In [193]:
from pycontractions import Contractions
from nltk import word_tokenize
#Peter Norvig 
from spellchecker import SpellChecker
from weighted_levenshtein import lev
import re
import numpy as np
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [2]:
contractions = Contractions('GoogleNews-vectors-negative300.bin.gz')
contractions.load_models()

In [194]:
raw_text = ["I'd like to know how I'd done that!",
                            "We're going to the zoo and I don't think I'll be home for dinner.",
                            "Unetek",
                            "Theyre going to the zoo and she'll be home for dinner."]

In [195]:
expanded_text=list(contractions.expand_texts(raw_text, precise=True))

In [196]:
tokenised_list = [word_tokenize(sentence) for sentence in expanded_text]

In [197]:
spell = SpellChecker(language='en',distance=1) 
spell.word_frequency.load_text_file('big.txt')

#spell.word_frequency.load_text_file('Europarl corpus')
spell.word_frequency.load_text('unitek')

In [198]:
corrected_list =[]
for sentence in tokenised_list:
    sent_list=[]
    for word in sentence:
        
        small_org_word= word.lower()
        correct_word =spell.correction(small_org_word)
        
        difference = lev(correct_word, small_org_word)/len(correct_word)
        
        if difference > 0.1:
            sent_list.append( correct_word)
        else:
            sent_list.append( word)
    corrected_list.append(sent_list)

In [199]:
untokenised_list = [TreebankWordDetokenizer().detokenize(sentence) for sentence in corrected_list]

In [200]:
decontracted_text = list(contractions.contract_texts(untokenised_list))

In [201]:
cer =[]
for org_text, new_text in zip(raw_text,decontracted_text):
    dist = levenshtein(new_text, org_text)
    length = len(org_text)
    cer.append(dist/length)
print("Mean CER = {}".format(100*np.mean(cer)))

Mean CER = 9.643874643874643
